In [56]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from statsmodels.stats.proportion import proportion_confint
from itertools import product

# Importing

In [57]:
parts_path = Path(r'E:\after_session_03_22\data\part_data')
os.listdir(parts_path)

['part1_dm_combined_18_95_J01.pkl',
 'part2_dm_combined_18_95_J01.pkl',
 'part3_dm_combined_18_95_J01.pkl',
 'part4_dm_combined_18_95_J01.pkl',
 'part5_dm_combined_18_95_J01.pkl',
 'part6_dm_combined_18_95_J01.pkl',
 'part7_dm_combined_18_95_J01.pkl',
 'part8_dm_combined_18_95_J01.pkl']

## Defining

In [58]:
start_date = {
    1: '2014-03-20',
    2: '2014-06-21',
    3: '2014-09-23',
    4: '2014-12-22',
    5: '2015-03-21',
    6: '2015-06-21',
    7: '2015-09-23',
    8: '2015-12-22',
    9: '2016-03-20',
    10: '2016-06-21',
    11: '2016-09-22',
    12: '2016-12-21',
}
end_date = {
    1: '2014-06-20',
    2: '2014-09-22',
    3: '2014-12-21',
    4: '2015-03-20',
    5: '2015-06-20',
    6: '2015-09-22',
    7: '2015-12-21',
    8: '2016-03-19',
    9: '2016-06-20',
    10: '2016-09-21',
    11: '2016-12-20',
    12: '2017-03-20',
}

# Calculating

In [59]:
to_groupby = ['member', 'season']

first_iter = True
for part_loc in os.listdir(parts_path):
    # Logging
    print('Processing:  ', part_loc)

    # Importing part
    part_loc = parts_path / part_loc
    part = pd.read_pickle(part_loc)

    # Filtering for ab drugs
    part = part.loc[part['ab']]

    # Building ab_class column
    part['ab_class'] = part['atc'].str[:4]

    # Adding season
    for t in range(1, 12 + 1):
        part.loc[
            (part['date'] >= start_date[t]) & (part['date'] <= end_date[t]),
            'season'
        ] = t
    part['season'] = part['season'].astype(int)

    # groupby
    if first_iter:
        df = part.groupby(to_groupby)['ab_class'].value_counts().to_frame().unstack(-1)
    first_iter = False
    df = pd.concat([
        df, part.groupby(to_groupby)['ab_class'].value_counts().to_frame().unstack(-1)
    ])


Processing:   part1_dm_combined_18_95_J01.pkl
Processing:   part2_dm_combined_18_95_J01.pkl
Processing:   part3_dm_combined_18_95_J01.pkl
Processing:   part4_dm_combined_18_95_J01.pkl
Processing:   part5_dm_combined_18_95_J01.pkl
Processing:   part6_dm_combined_18_95_J01.pkl
Processing:   part7_dm_combined_18_95_J01.pkl
Processing:   part8_dm_combined_18_95_J01.pkl


In [ ]:
df

## Adding cat from person_df_J01

In [ ]:
person_df_J01 = pd.read_pickle(r'E:\after_session_03_22\data\one_df\person_df_J01.pkl')
person_df_J01

In [78]:
# Merging
cols_to_merge = ['cat']

ab_class_trend = pd.merge(
    df['ab_class'].reset_index('season'),
    person_df_J01[cols_to_merge],
    how='left',
    on='member'
)

In [ ]:
ab_class_trend

In [81]:
# Dropping cat=0 because they where outliers.
outliers = (ab_class_trend.loc[ab_class_trend['cat'] == 0]).index
ab_class_trend.drop(outliers, axis=0, inplace=True)

## Saving

In [82]:
# Saving ab_class_trend
# ab_class_trend.to_pickle(r'E:\after_session_03_22\data\drug_class\ab_drug_class_numbers_seasonal.pkl')

## Building table for plotting

In [83]:
ab_class_trend = pd.read_pickle(r'E:\after_session_03_22\data\drug_class\ab_drug_class_numbers_seasonal.pkl')

In [ ]:
ab_class_trend

In [85]:
ab_classes = ['J01A', 'J01B', 'J01C', 'J01D', 'J01E', 'J01F', 'J01G', 'J01M', 'J01X']

In [ ]:
def table_builder(df):
    ab_table = pd.DataFrame(columns=)

In [88]:
trend_service_numbers = ab_class_trend.groupby(['cat', 'season'])[ab_classes].sum()

In [90]:
trend_service_numbers

J01A  J01B      J01C      J01D    J01E      J01F     J01G  \
cat season                                                                 
1   1         661.0   0.0   17868.0   13342.0   901.0    7809.0    979.0   
    2         754.0   1.0   13405.0   12523.0   897.0    5335.0    979.0   
    3         803.0   4.0   22609.0   14279.0   828.0    9379.0    936.0   
    4         932.0   7.0   28160.0   17334.0   815.0   13501.0   1033.0   
    5         657.0   3.0   17741.0   13658.0   844.0    8242.0    876.0   
    6         893.0   9.0   13553.0   12980.0  1024.0    5614.0    946.0   
    7         811.0   3.0   24633.0   15734.0   844.0   11377.0    965.0   
    8         915.0   4.0   24905.0   16816.0   685.0   13025.0    965.0   
    9         844.0   6.0   17835.0   14027.0   839.0    8808.0    911.0   
    10        889.0   3.0   14123.0   13559.0   887.0    6400.0    993.0   
    11        733.0   0.0   22284.0   15590.0   611.0   11632.0    869.0   
    12        805.0   0.0   19333.0   15362.0   568.0   10203.0    830.0   
2   1        2129.0   3.0   36792.0   28819.0  1669.0   15603.0   2038.0   
    2        2527.0   8.0   28582.0   27131.0  1674.0   11106.0   2124.0   
    3        2497.0   4.0   49194.0   32567.0  1575.0   19812.0   2033.0   
    4        2839.0  10.0   59357.0   40457.0  1473.0   28334.0   2297.0   
    5        2546.0   6.0   38060.0   30541.0  1505.0   17582.0   1968.0   
    6        2675.0   4.0   29449.0   29328.0  1818.0   12288.0   2223.0   
    7        2616.0   7.0   53488.0   36939.0  1576.0   25377.0   2090.0   
    8        2897.0   5.0   52962.0   39184.0  1315.0   27478.0   2070.0   
    9        2653.0   3.0   42134.0   34224.0  1697.0   20685.0   2118.0   
    10       2817.0   1.0   32259.0   31715.0  1763.0   14913.0   2093.0   
    11       2343.0   4.0   47364.0   35388.0  1273.0   25582.0   1757.0   
    12       2537.0   1.0   40594.0   32929.0  1097.0   22138.0   1721.0   
3   1        4252.0   7.0   59168.0   47691.0  2762.0   25629.0   3343.0   
    2        4776.0   1.0   46105.0   45052.0  2705.0   19113.0   3778.0   
    3        5114.0   7.0   81019.0   56583.0  2681.0   34025.0   3584.0   
    4        5723.0   6.0   94001.0   67617.0  2562.0   46067.0   3854.0   
    5        4916.0   6.0   62107.0   51741.0  2498.0   29398.0   3414.0   
    6        5402.0  12.0   48812.0   47934.0  2899.0   21392.0   3622.0   
    7        5235.0  11.0   86801.0   63375.0  2580.0   41873.0   3630.0   
    8        5491.0   9.0   84394.0   65036.0  2285.0   45141.0   3461.0   
    9        5543.0   5.0   71594.0   60095.0  2906.0   35759.0   3677.0   
    10       5865.0   1.0   55522.0   54425.0  2901.0   26099.0   3652.0   
    11       4278.0   4.0   76622.0   58774.0  2106.0   41982.0   2930.0   
    12       4546.0   0.0   65112.0   53339.0  1741.0   36245.0   2848.0   
4   1       10287.0  16.0  141087.0  123036.0  6838.0   60095.0   9172.0   
    2       11285.0   6.0  116702.0  120110.0  7282.0   49418.0   9976.0   
    3       11868.0  14.0  193011.0  147020.0  7130.0   79363.0   9988.0   
    4       13445.0  20.0  216840.0  170876.0  7402.0  101641.0  10765.0   
    5       12073.0  15.0  151867.0  137454.0  7033.0   70592.0   9792.0   
    6       12348.0  16.0  127270.0  132297.0  7678.0   56941.0  10673.0   
    7       11640.0  13.0  202606.0  157822.0  7078.0   93853.0   9910.0   
    8       12159.0  20.0  202798.0  165841.0  6921.0  100689.0  10069.0   
    9       13739.0   5.0  183492.0  164243.0  8197.0   88136.0  10908.0   
    10      13841.0  10.0  146081.0  147125.0  8084.0   68224.0  10615.0   
    11       9270.0   8.0  186960.0  147533.0  5950.0   92981.0   8336.0   
    12       9833.0   3.0  160828.0  134340.0  5289.0   81205.0   7742.0   

               J01M    J01X  
cat season                   
1   1        7142.0   182.0  
    2        7915.0   200.0  
    3        7223.0   201.0  
    4        7542.0   213.0  
    5        7382.0   202.0  
    

## Saving

In [91]:
# Saving trend_service_numbers
# trend_service_numbers.to_csv(r'E:\after_session_03_22\results\trend_service_numbers.csv')